<a href="https://colab.research.google.com/github/Swathi1309/ED18B034_ME18B133_Assignment1/blob/main/Assignment2/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv1D,Conv2D, BatchNormalization, Dropout, Activation, Softmax, MaxPooling2D
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam,RMSprop
import numpy as np

In [4]:
def CNN_wo_BN(X_train, filter_matrix, kernel_matrix, activation_matrix, fdropout, dense_no, learning_rate):

  # Input layer
  X_input = keras.Input(shape=X_train.shape)

  # Layer 1
  X = Conv2D(filter_matrix[0], kernel_matrix[0])(X_input)
  X = Activation(activation_matrix[0])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 2
  X = Conv2D(filter_matrix[1], kernel_matrix[1])(X)
  X = Activation(activation_matrix[1])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 3
  X = Conv2D(filter_matrix[2], kernel_matrix[2])(X)
  X = Activation(activation_matrix[2])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 4
  X = Conv2D(filter_matrix[3], kernel_matrix[3])(X)
  X = Activation(activation_matrix[3])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 5
  X = Conv2D(filter_matrix[4], kernel_matrix[4])(X)
  X = Activation(activation_matrix[4])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  X = Flatten()(X)
  X = Dense(dense_no,activation=activation_matrix[5])(X)
  X = Dropout(fdropout)(X)
  X = Dense(10,activation='softmax')(X)
  
  model=Model(inputs=X_input,outputs=X)
  model.complile(optimizer=Adam(learning_rate), loss='categorical_crossentropy',  metrics=['accuracy'])

  return model

In [ ]:
def CNN_w_BN(X_train, filter_matrix, kernel_matrix, activation_matrix, fdropout, dense_no, learning_rate):

  # Input layer
  X_input = keras.Input(shape=X_train.shape)

  # Layer 1
  X = Conv2D(filter_matrix[0], kernel_matrix[0]))(X_input)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[0])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 2
  X = Conv2D(filter_matrix[1], kernel_matrix[1])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[1])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 3
  X = Conv2D(filter_matrix[2], kernel_matrix[2])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[2])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 4
  X = Conv2D(filter_matrix[3], kernel_matrix[3])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[3])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 5
  X = Conv2D(filter_matrix[4], kernel_matrix[4])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[4])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  X = Flatten()(X)
  X = Dense(dense_no,activation=activation_matrix[5])(X)
  X = Dropout(fdropout)(X)
  X = Dense(10,activation='softmax')(X)
  
  model=Model(inputs=X_input,outputs=X)
  model.complile(optimizer=Adam(learning_rate), loss='categorical_crossentropy',  metrics=['accuracy'])

  return model

In [ ]:
def model_choose(batch_norm, highest_filter, filters_type, dropout):
  if filter_type == 'constant':
    filter_matrix = [filter_start]*4
  elif filter_type == 'doubling':
    filter_matrix = [highest_filter/8, highest_filter/4, highest_filter/2, highest_filter]
  else:
    filter_matrix = [highest_filter, highest_filter/2, highest_filter/4, highest_filter/8]
  wandb.log({"Filters": filter_matrix, "Dropout": dropout, "Batch_norm": batch_norm})

  if batch_norm = True:
    model = CNN_w_BN(X_train, filter_matrix, kernel_matrix, activation_matrix, dropout, dense_no)
  else:
    model = CNN_wo_BN(X_train, filter_matrix, kernel_matrix, activation_matrix, dropout, dense_no)
  
  return model


In [ ]:
@tf.custom_gradient
def GBRelu(x):
  def gradient(dy):
    grad = tf.cast(dy>0,"float32")*tf.cast(x>0, "float32")*dy
    return  grad
  return tf.nn.relu(x), gradient
# Here model is the best model
GBModel = Model(inputs = [model.inputs],outputs = [model.get_layer(index=8).output]) # Here the index is to be checked
layer_dict = [layer for layer in gb_model.layers[1:] if hasattr(layer,'activation')]
for layer in layer_dict:
  if layer.activation == tf.keras.activations.relu:
    layer.activation = GBRelu

with tf.GradientTape() as tape:
  inputs = tf.cast(X_input, tf.float32)
  tape.watch(inputs)
  outputs = GBModel(inputs)

grads = tape.gradient(outputs,inputs)[0]